# Validation

In [1]:
import math
import random

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import numpyro
import numpyro.distributions as dist
import pandas as pd
import scipy
from scipy import stats
import statsmodels.api as sm
import tqdm

from frugalCopyla.model import CopulaModel
from frugalCopyla import copula_functions as copula_lpdfs

/Users/danielmanela/.virtualenvs/causal-copulas/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
random.seed(0)

## Check seed in data generation works correctly

In [3]:
seed_input_dict = {
    'Z': {
        'dist': dist.Normal, 
        'formula': {'loc': 'A ~ 1', 'scale': 'A ~ 1'}, 
        'coeffs': {'loc': [0.], 'scale': [1.]}, 
        'link': None
    }, 
    'U': {
        'dist': dist.Normal, 
        'formula': {'loc': 'U ~ Z', 'scale': 'U ~ 1'}, 
        'coeffs': {'loc': [1., 2.], 'scale': [1.]}, 
        'link': None
    },     
    'X': {
        'dist': dist.Normal, 
        'formula': {'loc': 'X ~ Z', 'scale': 'X ~ 1'}, 
        'coeffs': {'loc': [0., 0.5], 'scale': [1.]}, 
        'link': None
    },
    'Y': {
        'dist': dist.Normal, 
        'formula': {'loc': 'Y ~ X', 'scale': 'Y ~ 1'}, 
        'coeffs': {'loc': [0., 1.], 'scale': [0.5]}, 
        'link': {'loc': None}
    },
    'copula': {
        'class': copula_lpdfs.multivar_gaussian_copula_lpdf, 
        'vars': ['Z', 'Y'], 
        'formula': {'rho_zy': 'c ~ Z'}, 
        'coeffs': {'rho_zy': [0.5, 1]}, 
        'link': {'rho_zy': jax.nn.sigmoid}
    }    
}

In [4]:
%%time
seed_mod = CopulaModel(seed_input_dict)
sim_data_seed = seed_mod.simulate_data(num_warmup=2000, num_samples=100_000, joint_status='continuous', seed=1)
sim_data_seed = pd.DataFrame(sim_data_seed)[['Z', 'U', 'X', 'Y']]
seed_df_1 = sim_data_seed.copy()

sim_data_seed = seed_mod.simulate_data(num_warmup=2000, num_samples=100_000, joint_status='continuous', seed=1)
sim_data_seed = pd.DataFrame(sim_data_seed)[['Z', 'U', 'X', 'Y']]

assert sim_data_seed.equals(seed_df_1)

sim_data_seed = seed_mod.simulate_data(num_warmup=2000, num_samples=100_000, joint_status='continuous', seed=2)
sim_data_seed = pd.DataFrame(sim_data_seed)[['Z', 'U', 'X', 'Y']]

assert ~sim_data_seed.equals(seed_df_1)

CPU times: user 13.7 s, sys: 192 ms, total: 13.9 s
Wall time: 13.8 s


## Example 1

In [4]:
input_dict = {
    'Z': {
        'dist': dist.Normal, 
        'formula': {'loc': 'A ~ 1', 'scale': 'A ~ 1'}, 
        'coeffs': {'loc': [0.], 'scale': [1.]}, 
        'link': None
    }, 
    'X': {
        'dist': dist.Normal, 
        'formula': {'loc': 'X ~ Z', 'scale': 'X ~ 1'}, 
        'coeffs': {'loc': [0., 0.5], 'scale': [1.]}, 
        'link': None
    },
    'Y': {
        'dist': dist.Normal, 
        'formula': {'loc': 'Y ~ X', 'scale': 'Y ~ 1'}, 
        'coeffs': {'loc': [0., 0.5], 'scale': [0.5]}, 
        'link': {'loc': None}
    }
}

In [7]:
cop_mod.parsed_model

{'Z': {'dist': numpyro.distributions.continuous.Normal,
  'formula': {'loc': 'A ~ 1', 'scale': 'A ~ 1'},
  'coeffs': {'loc': [0.0], 'scale': [1.0]},
  'link': {},
  'linear_predictor': {'loc': '0.0', 'scale': '1.0'}},
 'X': {'dist': numpyro.distributions.continuous.Normal,
  'formula': {'loc': "X ~ 1 + record_dict['Z']", 'scale': 'X ~ 1'},
  'coeffs': {'loc': [0.0, 0.5], 'scale': [1.0]},
  'link': {},
  'linear_predictor': {'loc': "0.0 + 0.5 * record_dict['Z']", 'scale': '1.0'}},
 'Y': {'dist': numpyro.distributions.continuous.Normal,
  'formula': {'loc': "Y ~ 1 + record_dict['X']", 'scale': 'Y ~ 1'},
  'coeffs': {'loc': [0.0, 0.5], 'scale': [0.5]},
  'link': {'loc': None},
  'linear_predictor': {'loc': "0.0 + 0.5 * record_dict['X']", 'scale': '0.5'}}}

Preparing the `CopulaModel`:

In [6]:
%%time
cop_mod = CopulaModel(input_dict)
sim_data = cop_mod.simulate_data(num_warmup=2000, num_samples=1_000_000, joint_status='continuous', seed=1)
sim_data = pd.DataFrame(sim_data)[['Z', 'X', 'Y']]
sim_data.describe()

INFO: No copula specified.
CPU times: user 4.47 s, sys: 61.6 ms, total: 4.54 s
Wall time: 4.53 s


,Z,X,Y
count,1000000.000000,1000000.000000,1000000.000000
mean,0.000071,0.001043,0.000600
std,0.998562,1.115881,0.749170
min,-4.583462,-5.911977,-3.444071
25%,-0.673928,-0.752323,-0.505126
50%,-0.000857,0.000128,0.000728
75%,0.674170,0.753669,0.506009
max,4.797225,5.322031,3.405338


In [7]:
assert stats.kstest(sim_data[['Z']].values.ravel(), stats.norm.cdf).pvalue > 0.1

In [8]:
assert (stats.kstest(
    (sim_data[['X']].values.ravel() - .5 * sim_data[['Z']].values.ravel()), 
    stats.norm.cdf
).pvalue > 0.1)

In [9]:
lm = sm.OLS(sim_data[['X']].values.ravel(), sm.add_constant(sim_data[['Z']]))
lm_results = lm.fit()

summary = pd.concat([ 
    pd.DataFrame(lm_results.params).rename(columns={0: 'estimate'}),
    pd.DataFrame(lm_results.bse * 2).rename(columns={0: '2sd'})
], axis=1)
summary['true_vals'] = [0, 0.5]
summary['true_estimate'] = (
    (summary['estimate'] + summary['2sd'] > summary['true_vals']) & (summary['estimate'] - summary['2sd'] < summary['true_vals'])
)
display(summary)
assert summary.loc['const', 'true_estimate'] == True
assert summary.loc['Z', 'true_estimate'] == True

,estimate,2sd,true_vals,true_estimate
const,0.001008,0.001997,0.0,True
Z,0.498644,0.002000,0.5,True


In [10]:
lm = sm.OLS(sim_data[['Y']].values.ravel(), sm.add_constant(sim_data[['Z', 'X']]))
lm_results = lm.fit()

summary = pd.concat([ 
    pd.DataFrame(lm_results.params).rename(columns={0: 'estimate'}),
    pd.DataFrame(lm_results.bse * 2).rename(columns={0: '2sd'})
], axis=1)
summary['true_vals'] = [0, 0, 0.5]
summary['true_estimate'] = (
    (summary['estimate'] + summary['2sd'] > summary['true_vals']) & (summary['estimate'] - summary['2sd'] < summary['true_vals'])
)
display(summary)
assert summary.loc['const', 'true_estimate'] == True
assert summary.loc['Z', 'true_estimate'] == True

,estimate,2sd,true_vals,true_estimate
const,0.000079,0.001000,0.0,True
Z,0.000074,0.001119,0.0,True
X,0.499932,0.001001,0.5,True


## Example 2

In [11]:
input_dict_2 = {
    'Z': {
        'dist': dist.Normal, 
        'formula': {'loc': 'A ~ 1', 'scale': 'A ~ 1'}, 
        'coeffs': {'loc': [0.], 'scale': [1.]}, 
        'link': None
    }, 
    'U': {
        'dist': dist.Normal, 
        'formula': {'loc': 'U ~ 1', 'scale': 'U ~ 1'}, 
        'coeffs': {'loc': [0.], 'scale': [1.]}, 
        'link': None
    },     
    'X': {
        'dist': dist.BernoulliProbs, 
        'formula': {'probs': 'X ~ Z'}, 
        'coeffs': {'probs': [0., 0.5]}, 
        'link': {'probs': jax.scipy.special.expit}
    },
    'Y': {
        'dist': dist.Normal, 
        'formula': {'loc': 'Y ~ X', 'scale': 'Y ~ 1'}, 
        'coeffs': {'loc': [0., 1.], 'scale': [0.5]}, 
        'link': {'loc': None}
    },
    'copula': {
        'class': copula_lpdfs.multivar_gaussian_copula_lpdf, 
        'vars': ['Z', 'Y'], 
        'formula': {'rho_zy': 'c ~ Z'}, 
        'coeffs': {'rho_zy': [0.5, 1]}, 
        'link': {'rho_zy': jax.nn.sigmoid}
    }    
}

In [17]:
%%time
cop_mod_2 = CopulaModel(input_dict_2)
sim_data_2 = cop_mod_2.simulate_data(num_warmup=2000, num_samples=1_000_000, joint_status='mixed', seed=0)
sim_data_2 = pd.DataFrame(sim_data_2)[['Z', 'X', 'Y']]

INFO: No copula specified.
CPU times: user 4.76 s, sys: 58.7 ms, total: 4.82 s
Wall time: 4.81 s


In [18]:
lm = sm.GLM(sim_data_2[['X']].values.ravel(), sm.add_constant(sim_data_2[['Z']]), family=sm.families.Binomial())
lm_results = lm.fit()

summary = pd.concat([ 
    pd.DataFrame(lm_results.params).rename(columns={0: 'estimate'}),
    pd.DataFrame(lm_results.bse * 2).rename(columns={0: '2sd'})
], axis=1)
summary['true_vals'] = [0, 0.5]
summary['true_estimate'] = (
    (summary['estimate'] + summary['2sd'] > summary['true_vals']) & (summary['estimate'] - summary['2sd'] < summary['true_vals'])
)
display(summary)
assert summary.loc['const', 'true_estimate'] == True
assert summary.loc['Z', 'true_estimate'] == True

,estimate,2sd,true_vals,true_estimate
const,0.001899,0.004115,0.0,True
Z,0.496341,0.004343,0.5,True


In [19]:
lm = sm.OLS(sim_data_2[['Y']].values.ravel(), sm.add_constant(sim_data_2[['Z', 'X']]))
lm_results = lm.fit()

summary = pd.concat([ 
    pd.DataFrame(lm_results.params).rename(columns={0: 'estimate'}),
    pd.DataFrame(lm_results.bse * 2).rename(columns={0: '2sd'})
], axis=1)
summary['true_vals'] = [0, 0.5, 1]
summary['true_estimate'] = (
    (summary['estimate'] + summary['2sd'] > summary['true_vals']) & (summary['estimate'] - summary['2sd'] < summary['true_vals'])
)
display(summary)
assert summary.loc['const', 'true_estimate'] == True
assert summary.loc['X', 'true_estimate'] == True

,estimate,2sd,true_vals,true_estimate
const,-0.000735,0.001437,0.0,True
Z,-0.000347,0.001028,0.5,False
X,1.000068,0.002060,1.0,True


### Check that binomial works OK

In [20]:
input_dict_3 = {
    'Z': {
        'dist': dist.BernoulliProbs, 
        'formula': {'probs': 'A ~ 1'}, 
        'coeffs': {'probs': [0.5]}, 
        'link': None
    }, 
    'X': {
        'dist': dist.Normal, 
        'formula': {'loc': 'X ~ Z', 'scale': 'X ~ 1'}, 
        'coeffs': {'loc': [0., 0.5], 'scale': [1.]}, 
        'link': None
    },
    'Y': {
        'dist': dist.Normal, 
        'formula': {'loc': 'Y ~ X', 'scale': 'Y ~ 1'}, 
        'coeffs': {'loc': [0., 1.], 'scale': [0.5]}, 
        'link': None
    }   
}

In [21]:
%%time
cop_mod_3 = CopulaModel(input_dict_3)
sim_data_3 = cop_mod_3.simulate_data(num_warmup=5000, num_samples=1_000_000, joint_status='mixed', seed=0)
sim_data_3 = pd.DataFrame(sim_data_3)[['Z', 'X', 'Y']]
sim_data_3.describe()

INFO: No copula specified.
CPU times: user 4.52 s, sys: 58.2 ms, total: 4.58 s
Wall time: 4.57 s


,Z,X,Y
count,1000000.000000,1000000.000000,1000000.000000
mean,0.499614,0.248476,0.248497
std,0.500000,1.027565,1.142574
min,0.000000,-4.343137,-4.711799
25%,0.000000,-0.445188,-0.522827
50%,0.000000,0.247508,0.246190
75%,1.000000,0.941066,1.018002
max,1.000000,5.440841,5.630909


In [22]:
lm = sm.OLS(sim_data_3[['X']].values.ravel(), sm.add_constant(sim_data_3[['Z']]))
lm_results = lm.fit()

summary = pd.concat([ 
    pd.DataFrame(lm_results.params).rename(columns={0: 'estimate'}),
    pd.DataFrame(lm_results.bse * 2).rename(columns={0: '2sd'})
], axis=1)
summary['true_vals'] = [0, 0.5]
summary['true_estimate'] = (
    (summary['estimate'] + summary['2sd'] > summary['true_vals']) & (summary['estimate'] - summary['2sd'] < summary['true_vals'])
)
display(summary)
assert summary.loc['const', 'true_estimate'] == True
assert summary.loc['Z', 'true_estimate'] == True

,estimate,2sd,true_vals,true_estimate
const,-0.000275,0.002819,0.0,True
Z,0.497887,0.003988,0.5,True


In [23]:
lm = sm.OLS(sim_data_3[['Y']].values.ravel(), sm.add_constant(sim_data_3[['Z', 'X']]))
lm_results = lm.fit()

summary = pd.concat([ 
    pd.DataFrame(lm_results.params).rename(columns={0: 'estimate'}),
    pd.DataFrame(lm_results.bse * 2).rename(columns={0: '2sd'})
], axis=1)
summary['true_vals'] = [0, 0., 1]
summary['true_estimate'] = (
    (summary['estimate'] + summary['2sd'] > summary['true_vals']) & (summary['estimate'] - summary['2sd'] < summary['true_vals'])
)
display(summary)
assert summary.loc['const', 'true_estimate'] == True
assert summary.loc['X', 'true_estimate'] == True

,estimate,2sd,true_vals,true_estimate
const,-0.000094,0.001413,0.0,True
Z,0.000293,0.002060,0.0,True
X,0.999875,0.001003,1.0,True


In [24]:
assert stats.chisquare(sim_data_3[['Z']].values.ravel()).pvalue > 0.1